In [1]:
import bs4
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from urllib.request import urlopen as uReq
import pandas as pd
import requests
import re

In [2]:
url1 = "https://coinmarketcal.com/en/?page="
url2 = "https://coinmarketcal.com/en/coin/"
pageNo = 1
lastProj = ""
proj = []
my_dict={"Project" : [], "Date" : [], "Url" : []}
cond = True
while (cond):
    url = url1 + str(pageNo)
    uClient = uReq(url)
    page_html = uClient.read()
    uClient.close()
    req = requests.get(url)
    page_soup = soup(page_html, "html.parser")
    project_cards = page_soup.findAll("div",{"class": "card__body"})
        
    for i in range(0, len(project_cards)):
        url__ = str(project_cards[i]('a', attrs={'href': re.compile("^/en/coin")}))
        token_name = re.search('/en/coin/(.*?)">', url__)
        if token_name == None:
            continue
        name = project_cards[i]("h5",{"class":"card__title mb-0 ellipsis"})[0].text
        _date = project_cards[i]("span", {"class" : "ellipsis ml-2"})
        __date = project_cards[i]("h5",{"class" : "card__date mt-0"})
        token_url = str(token_name.group(1))
        real_url = url2 + token_url
        date = ""
        if(len(_date) > 0): date = _date[0].text
        if(len(__date) > 0): date = __date[0].text
        proj.append({"project": name, "date": date, "url": real_url})
                
    if(lastProj== proj[-1]["project"]):
        cond = False
        break

    lastProj = name
    pageNo += 1

df = pd.DataFrame(proj)
df = (df.drop_duplicates(subset = "project"))
print(df["project"].count())
print(df)




1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
470
                   project                       date  \
0        2 upcoming events                              
1     AMA with HyperSphere                03 Jan 2022   
2       V1 Platform Launch                03 Jan 2022   
3          BitMart Listing                03 Jan 2022   
5            LBank Listing                03 Jan 2022   
..                     ...                        ...   
556          DAPP Platform  30 Sep 2023 (or earlier)    
557  Klever Blockchain 3.0  31 Dec 2023 (or earlier)    
558   Klever Privacy Phone  31 Mar 2024 (or earlier)    
559  Klever Blockchain 4.0  31 Mar 2024 (or earlier)    
562   Monthly Airdrop Ends                01 Aug 2024   

                                                   url  
0    https://coinmarketcal.com/en/coin/harmony" tit...  
1              https://coinmarketcal.com/en/coin/verge  
2            https://coinmarketcal.com/en

In [3]:
Mainnet = df[df['project'].str.contains('Mainnet')]
Testnet = df[df['project'].str.contains('Testnet')]
Audit = df[df['project'].str.contains('Audit')]
writer = pd.ExcelWriter('Data Extraction.xlsx', engine='xlsxwriter')
Mainnet.to_excel(writer, sheet_name='Mainnet')
Testnet.to_excel(writer, sheet_name='Testnet')
Audit.to_excel(writer, sheet_name='Audit')
writer.save()